In [ ]:
import torch
import os
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn as nn
import numpy as np

import seaborn as sns

device = 'cuda'

In [ ]:
fs= 250                  #sampling frequency采样频率
channel= 22              #number of electrode
num_input= 1             #输入通道数（eeg只有一个）
num_class= 5             #number of classes 模型需要分类的目标分类数
signal_length = 1000      #number of sample in each tarial每个样本包含的数据点

F1= 8                    #number of temporal filters时间滤波器的数量
D= 3                     #depth multiplier (number of spatial filters)深度乘数（空间滤波器数量）
F2= D*F1                 #number of pointwise filters点乘滤波器的数量

In [ ]:
kernel_size_1= (1,round(fs/2))  #时间维度 跨度为0.5s
kernel_size_2= (channel, 1) #空间维度 每次计算所有通道
kernel_size_3= (1, round(fs/8)) #时间维度 跨度为0.125s
kernel_size_4= (1, 1)

kernel_avgpool_1= (1,4) #平均池化层的池化核大小
kernel_avgpool_2= (1,8)
dropout_rate= 0.2 #防止过拟合

#卷积核填充（padding）,保证输入和输出尺寸匹配
ks0= int(round((kernel_size_1[0]-1)/2))
ks1= int(round((kernel_size_1[1]-1)/2))
kernel_padding_1= (ks0, ks1-1)
ks0= int(round((kernel_size_3[0]-1)/2))
ks1= int(round((kernel_size_3[1]-1)/2))
kernel_padding_3= (ks0, ks1)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Squeeze-and-Excitation (SE) Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y)
        return x * y

# EEGNet Model with SE and Bidirectional LSTM
class EEGNetWithSE(nn.Module):
    def __init__(self):
        super(EEGNetWithSE, self).__init__()

        # Layer 1: Initial convolution and batch normalization
        self.conv2d = nn.Conv2d(num_input, F1, kernel_size_1, padding=kernel_padding_1)
        self.Batch_normalization_1 = nn.BatchNorm2d(F1)

        # Multi-scale convolution layers (three different kernel sizes)
        self.conv2d_small = nn.Conv2d(F1, F1, kernel_size=(1, 3), padding=(0, 1))   # Small kernel
        self.conv2d_medium = nn.Conv2d(F1, F1, kernel_size=(1, 5), padding=(0, 2))  # Medium kernel
        self.conv2d_large = nn.Conv2d(F1, F1, kernel_size=(1, 7), padding=(0, 3))   # Large kernel
        
        # Batch normalization for multi-scale outputs
        self.Batch_normalization_multiscale = nn.BatchNorm2d(F1 * 3)
        self.relu = nn.ReLU()

        # Layer 2: Depthwise convolution
        self.Depthwise_conv2D = nn.Conv2d(F1 * 3, F1 * 3, kernel_size_2, groups=F1 * 3)
        self.Batch_normalization_2 = nn.BatchNorm2d(F1 * 3)
        self.Elu = nn.ELU()
        self.Average_pooling2D_1 = nn.AvgPool2d(kernel_avgpool_1)
        self.Dropout = nn.Dropout2d(dropout_rate)

        # Layer 3: Separable convolution
        self.Separable_conv2D_depth = nn.Conv2d(F1 * 3, F1 * 3, kernel_size_3, padding=kernel_padding_3, groups=F1 * 3)
        self.Separable_conv2D_point = nn.Conv2d(F1 * 3, F2, kernel_size_4)
        self.Batch_normalization_3 = nn.BatchNorm2d(F2)
        self.Average_pooling2D_2 = nn.AvgPool2d(kernel_avgpool_2)

        # **Squeeze-and-Excitation Layer (added)**
        self.se_block = SEBlock(F2)

        # LSTM layer (added)
        self.lstm_hidden_size = 128
        # Modify to use Bidirectional LSTM
        self.lstm = nn.LSTM(F2, self.lstm_hidden_size, batch_first=True, bidirectional=True)

        # Layer 4: Flatten and fully connected layers
        self.Flatten = nn.Flatten()
        # Fixed: Since we're using bidirectional LSTM, output size becomes 2 * lstm_hidden_size
        self.Dense = nn.Linear(self.lstm_hidden_size * 2, num_class)  # *2 for bidirectional LSTM
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Layer 1: Initial convolution + batch normalization
        y = self.Batch_normalization_1(self.conv2d(x))

        # Multi-scale convolution layer: Apply three different convolutions
        y_small = self.conv2d_small(y)   # Small kernel
        y_medium = self.conv2d_medium(y)  # Medium kernel
        y_large = self.conv2d_large(y)   # Large kernel
        
        # Concatenate the three outputs along the channel dimension
        y = torch.cat((y_small, y_medium, y_large), dim=1)
        
        # Apply batch normalization and ReLU
        y = self.relu(self.Batch_normalization_multiscale(y))

        # Layer 2: Depthwise convolution + batch normalization + activation
        y = self.Batch_normalization_2(self.Depthwise_conv2D(y))
        y = self.Elu(y)
        y = self.Dropout(self.Average_pooling2D_1(y))

        # Layer 3: Separable convolution + batch normalization + activation
        y = self.Separable_conv2D_depth(y)
        y = self.Batch_normalization_3(self.Separable_conv2D_point(y))
        y = self.Elu(y)
        y = self.Dropout(self.Average_pooling2D_2(y))

        # **Squeeze-and-Excitation Module**
        y = self.se_block(y)  # Apply SE block

        # LSTM layer
        y = y.permute(0, 2, 3, 1)  # Change the shape to (batch_size, time_steps, channels)
        y = y.reshape(x.size(0), -1, F2)  # Reshape to match LSTM input format (batch_size, time_steps, features)

        # Get the LSTM output
        y, (h_n, c_n) = self.lstm(y)  # h_n: (num_layers * num_directions, batch_size, lstm_hidden_size)

        # Extract the last hidden state from the bidirectional LSTM
        y = torch.cat((h_n[0], h_n[1]), dim=-1)  # Concatenate the hidden states of both directions (h_n[0] and h_n[1])

        # Layer 4: Flatten and fully connected layer
        y = self.Dense(y)  # Linear layer to match output dimensions
        y = self.Softmax(y)

        return y

# 初始化模型和优化器
model = EEGNetWithSE().to(device)  # 初始化模型并确保它在 GPU 上
